In [ ]:
#use this command to install the required packages 
!pip install -r requirements.txt 

In [ ]:
#install these packages to get the resources you need
import pandas as pd
import numpy as np
import requests
import dask as dd
import seaborn as sns

In [ ]:
%%time 

#here is the link to the file in Google Drive
#V1 data
#url_raw='https://drive.google.com/file/d/1_mrcfR02LP_anbtdYq0Q1YuRN5-JCHE1/view?usp=sharing'


# url_raw = 'https://drive.google.com/file/d/13vj4YxKRs1Tw1VPGhTefBxxKQuC5pRd_/view?usp=sharing'
# #url_raw_mn = 'https://drive.google.com/file/d/1YRh58UiKWNtxfvYa0IzFF3oYL42iBs7d/view?usp=sharing'
# req = requests.get(url_raw)
# url_content = req.content
# with open('test_data.csv',encoding="utf-8") as file:
#     df=pd.read_csv(file,delimiter=',')
#========Straight loading the data==============
df=pd.read_csv('test_data_v1.csv',delimiter=',')

#===========Directly from my laptop============
# tab=pd.read_csv('test_data_v1.csv',delimiter=',',iterator=True,chunksize=200000)
# df=pd.concat(tab)
    



# req_mn = requests.get(url_raw_mn)
# url_content_mn = req_mn.content
# with open('test_data_mn.csv',) as file_mn:
#     df_mn=pd.read_csv(file_mn,delimiter=',')

In [ ]:
df.columns

In [ ]:
#uncode the education;gender;race columns with targetsmart data 
#1. load targetsmart data
target_smart=pd.read_excel('targetsmart_data_dictionary_q2_2021.xlsx',sheet_name='Copy of Raw TargetSmart Field V',header=2)

In [ ]:
df.columns
#target_smart.columns=target_smart.iloc[1]
target_smart.drop(target_smart.index[[0,1]],inplace=True)
target_smart.head()

In [ ]:
#grouping the data.


df.groupby(['vb_tsmart_zip'])['vb_tsmart_zip'].count()
#p_summ = p.groupby( attributes_list, as_index=False ).agg( {'AMT':sum })


In [ ]:
#look at the properties of the two datasets 
#print(df.head())
print(df.shape)


# print(df_mn.head())
# print(df_mn.shape)

# Data cleaning for Human Capital Project

First Goal: Create a column that identifies whether or not someone works in tech based upon a) their occupation status and/or b) working for/owning a company with a SIC code in the technology sector (i.e., any org with a code in the office of technology) 

In [ ]:
# Data cleaning for test mn data, but can be motified for larger dataset (do not want to crash my laptop while testing)
##Need to create a binary column that identifies if someone is in tech
df.groupby('vb_occupation')['vb_occupation'].count()

In [ ]:
df['vb_occupation'].unique()

In [ ]:
#Other occupation marker
df.groupby('tb_occupation_cd')['tb_occupation_cd'].count()

In [ ]:
df['tb_occupation_cd'].unique()

In [ ]:
#compare both occupations columns
df['Occupation_Match'] = np.where(df['vb_occupation']==df['tb_occupation_cd'], 
                                           'yes', 'no')

In [ ]:
df.groupby('tb_boah_biz_sic_cd')['tb_boah_biz_sic_cd'].count()
#834 different codes

In [ ]:
df['tb_boah_biz_sic_cd'].astype(int, errors = 'ignore').astype(str).split('.')

In [ ]:
df.groupby('tb_boah_biz_sic_2_cd')['tb_boah_biz_sic_2_cd'].count()
#89 different codes

In [ ]:
# load in file with SIC codes
sic_codes=pd.read_excel('SIC_Codes_5.31.2021.xlsx')
sic_codes.columns

#make a dictionary (First attempt)
sic_dict=sic_codes.to_dict('records')
sic_dict

In [ ]:
# antoher type of dictionary that may be easier to manuver
sic_dict2=sic_codes.set_index('SIC Code').to_dict()['Office']
sic_dict2

In [ ]:
#in the dataframe SIC Code is #.0 so the column needs to be clean
list_SIC_occ=[]
for occ_val in df['tb_boah_biz_sic_cd']:
    clean_val=str(occ_val).split('.')[0]
    list_SIC_occ.append(clean_val)
    
# add column with values

df['SIC_code']=df['tb_boah_biz_sic_cd'].map(sic_dict2)
df.head()
df['SIC_code'].unique()

In [ ]:
#dictionary for occupation
occ_codes=target_smart['Field']=='vb.occupation'
occ_df=target_smart[occ_codes]
occ_df.drop(occ_df.index[[0]],inplace=True)
occ_df.head()

In [ ]:
occ_dict=occ_df.set_index('Field Value').to_dict()['Value Description']

In [ ]:
df['Occupation_vb']=df['vb_occupation'].map(occ_dict)
df['Occupation_vb'].unique()
#'Professional/Technical', 'Medical/Health Services',
#        'Sales/Service', 'Administrative/Management', 'Financial Services',
#        'Skilled Trade', 'Upper Management/Executive', 'Office/Clerical',
#        'Self Employed', 'Farming/Agriculture', 'Military',
#        'Teacher/Educator', 'Legal Services', 'Religious'

In [ ]:
df['Occupation_tb']=df['tb_occupation_cd'].map(occ_dict)
df['Occupation_tb'].unique()
# 'Office/Clerical', nan, 'Medical/Health Services',
#        'Professional/Technical', 'Skilled Trade', 'Farming/Agriculture',
#        'Upper Management/Executive', 'Sales/Service', 'Military',
#        'Legal Services', 'Religious', 'Self Employed',
#        'Financial Services', 'Teacher/Educator',
#        'Administrative/Management

Second Goal: Create a column with a binary code for “tech_hub_zip9” that identifies all zip codes that are associated with the current tech hubs: New York, Boston, Seattle, San Francisco, and San Jose

In [ ]:
tech_zip=pd.read_excel('Tech_ZipCodes.xlsx')
tech_zipl=tech_zip['Zip Code'].to_list()

tech_zipl


In [ ]:
zip_bin=[]
tech_ziplist=tech_zipl # this is a test list, but once we finalize a zip code list we can
for zip_code in df['vb_tsmart_zip']:
    if zip_code in tech_ziplist:
        zip_code=1
        zip_bin.append(zip_code)
    else:
        zip_code=0
        zip_bin.append(zip_code)
        
zip_bin

In [ ]:
#create new column with this
df['tech_hub_zip9']=zip_bin

Third Goal: Review coding for education, gender, and race to make sure ACS coding maps to coding in Targetsmart data.

In [ ]:
#2 dictionary for race
race_codes=target_smart['Field']=='vb.voterbase_race'
race_df=target_smart[race_codes]
race_df.head()

In [ ]:
#3 dictionary for gender
gender_codes=target_smart['Field']=='vb.voterbase_gender'
gender_df=target_smart[gender_codes]
gender_df.head()

In [ ]:
#4 dictionary for education
edu_codes=target_smart['Field']=='vb.education'
edu_df=target_smart[edu_codes]
edu_df.drop(edu_df.index[[0]],inplace=True)
edu_df.head()

In [ ]:
#edu dictionary
edu_dict=edu_df.set_index('Field Value').to_dict()['Value Description']
edu_dict

In [ ]:
#created a categorical column for education
df['Education']=df['vb_education'].map(edu_dict)
df.head()

In [ ]:
df.columns

In [ ]:
df['tb_college_age_flg'].replace({1.: '18-22 years old'}, inplace=True)
df['tb_starting_out_flg'].replace({1.: '23-30 years old'}, inplace=True)
df['tb_career_bldr_flg'].replace({1.: '31-45 years old'}, inplace=True)
df['tb_prime_earner_flg'].replace({1.: '46-64 years old'}, inplace=True)
df['tb_retiree_flg'].replace({1.: '65 years or older'}, inplace=True)



In [ ]:
college_flag=target_smart['Field']=='tb.prime_earner_flg'
college_df=target_smart[college_flag]
college_df['Field Description'].unique()

In [ ]:
#insert table
#did not have enough memory
cols=['tb_college_age_flg', 'tb_starting_out_flg', 'tb_career_bldr_flg',
       'tb_prime_earner_flg', 'tb_retiree_flg']
df["Age Buckets"] = df[cols].apply(lambda x: ','.join(x.dropna()), axis=1)


END of First Data Clean Up 6.6.2021

**Only use data from POC talent**

In [ ]:
df.groupby('vb_voterbase_race')['vb_voterbase_race'].count()

In [ ]:
df_diverse=df.loc[~((df['vb_voterbase_race']=='Caucasian') |(df['vb_voterbase_race']=='Uncoded')|(df['vb_voterbase_race']=='Other')|(df['vb_voterbase_race']=='Asian')) & (df['Occupation']=='Professional/Technical') ]
df_diverse.shape

**Sample Heat Map for Minnesota**

In [ ]:
#libraties needed for heatmap
!pip install folium
import numpy                 as np
import pandas                as pd
import matplotlib.pyplot     as plt
import seaborn               as sns
import folium  #needed for interactive map
from folium.plugins import HeatMap
%matplotlib inline
sns.set()

In [ ]:
#Zoom into area
ny_coords = [46, -94] # lat, long


In [ ]:
#heat map
folium_hmap = folium.Map(location=ny_coords, zoom_start=6, tiles="CartoDB positron")

In [ ]:
#zip coordinates from diverse dataset
my_zip = zip(df_diverse['vb_tsmart_latitude'], df_diverse['vb_tsmart_longitude'])
list_of_my_zip = list(my_zip)
list_of_my_zip[0:15]
len(list_of_my_zip)

In [ ]:
hm_layer = HeatMap(list_of_my_zip,
                   # These are parameters that we tweak manually to adjust color
                   # See folium docs for more information
                   min_opacity=0.2,
                   radius=8,
                   blur=6, 
                 )

In [ ]:
folium_hmap.add_child(hm_layer)
folium_hmap

## EDA for Salary

In [ ]:
df['hh_income'].unique()
#there seems to be 5 bins for household income 
#1- less than 40K
#2- between 40K-99K
#3- between 100K-140K
#4- between 140K-200K
#5- more than 200K

df['tb_income_amt']
#income listed as ##.0 for individuals

In [ ]:
income_analysis=df.groupby(['vb_voterbase_race','Education','hh_income'])['hh_income'].count()#household income
income_analysis=income_analysis.to_frame()
income_analysis=income_analysis.rename(columns={'hh_income':'raw count'})
income_analysis=income_analysis.reset_index()
income_analysis

In [ ]:
#bar plot to analyze this data
import seaborn as sns
g = sns.FacetGrid(income_analysis, col="Education",size=6, aspect=1,margin_titles=True,col_wrap=2,sharey=False,sharex=False, row_order=['Less than High School Diploma','Some College','High School','College Graduate','Post Graduate','Unknown'])
g.map(sns.barplot, "vb_voterbase_race", "raw count", 'hh_income', order=['Caucasian', 'Uncoded', 'African-American', 'Asian',
       'Native American', 'Hispanic', 'Other'])
g.add_legend()
g.fig.subplots_adjust(top=0.9)
g.fig.suptitle('Education and Household Income of All Targetsmart Data')
g.set_xticklabels(rotation=-10,fontsize=8)
#g.savefig('education.png')




Look at how data is different by education and occupation[tech]

In [ ]:
#look at tech jobs:
tech=df[df['Occupation']=='Professional/Technical']
tech.columns
#df.columns

In [ ]:
#only technical jobs from the large dataset
income_occ_analysis=tech.groupby(['vb_voterbase_race','Education','hh_income'])['hh_income'].count()
income_occ_analysis=income_occ_analysis.to_frame()
income_occ_analysis=income_occ_analysis.rename(columns={'hh_income':'raw count'})
income_occ_analysis=income_occ_analysis.reset_index()
income_occ_analysis.columns

In [ ]:
g = sns.FacetGrid(income_occ_analysis, col="Education",size=6, aspect=1,margin_titles=True,col_wrap=2,sharey=False,sharex=False)
g.map(sns.barplot, "vb_voterbase_race", "raw count", 'hh_income', order=['Caucasian', 'Uncoded', 'African-American', 'Asian',
       'Native American', 'Hispanic', 'Other'])
g.add_legend()
g.set_xticklabels(rotation=-10,fontsize=8)
g.fig.subplots_adjust(top=0.9)
g.fig.suptitle('Technology Education and Household Income of All Targetsmart Data')
g.savefig('education_tech.png')

In [ ]:
#income by state
income_state_analysis=tech[['vb_voterbase_race','vb_tsmart_state','tb_income_amt']]
income_state_analysis

In [ ]:
g = sns.FacetGrid(income_state_analysis, col="vb_tsmart_state",size=6, aspect=1,margin_titles=True,col_wrap=5,sharey=False,sharex=False)
g.map(sns.boxplot, "vb_voterbase_race", "tb_income_amt", order=['Caucasian', 'Uncoded', 'African-American', 'Asian',
       'Native American', 'Hispanic', 'Other'])
g.add_legend()
g.set_xticklabels(rotation=-10,fontsize=8)
g.savefig('Income_tech_bystate.png')

In [ ]:
income_state_analysis.groupby(['vb_tsmart_state','vb_voterbase_race'])['tb_income_amt'].describe()

In [ ]:
df.to_csv('test_data_clean_v2.csv')